In [1]:
import netCDF4 as nc
import xarray as xr
import numpy as np
import os

# ACTRIS NRT

In [2]:
files = [
    'RO0010R.20240714060000.20241014053408.aerosol_mass_spectrometer...3mo.1h.RO03L_Aerodyne_Q-ACSM_140-172_NRT.RO03L_Aerodyne_Q-ACSM_.lev1.5.nc',
    'RO0010R.20240714050000.20241014050803.filter_absorption_photometer...3mo.1h.RO03L_Magee_AE33_INO_NRT.RO03L_eBC_v1.lev3b.nc',
    'RO0010R.20240714050000.20241014050407.filter_absorption_photometer...3mo.1h.RO03L_Magee_AE33_INO_NRT.RO03L_AE33.lev1.5.nc',
    'NO0002R.20241002110000.20241004105104.cpc...47h.1h.NO01L_TSI_3750_BIR_NRT.NO01L_CPC_acquisition_lev0_0_0_1.lev1.5.nc',
    'NO0002R.20241002090000.20241009001220.filter_absorption_photometer...1w.1h.NO01L_Magee_AE33_BIR_dry_NRT.NO01L_abs_coef_AE33_v1.lev1.5.nc',
    'NO0002R.20240715070000.20241014043606.aerosol_mass_spectrometer...13w.1h.NO01L_Aerodyne_Q-ACSM_140-144_NRT.NO01L_Aerodyne_Q-ACSM_BIR.lev1.5.nc',
    'IT0022C.20240714050000.20241014051210.filter_absorption_photometer...3mo.1h.IT06L_Magee_AE33_BO_NRT.IT06L_eBC_v1.lev3b.nc',
    'IT0022C.20240714050000.20241014050605.filter_absorption_photometer...3mo.1h.IT06L_Magee_AE33_BO_NRT.IT06L_AE33.lev1.5.nc',
    'IT0009R.20240713170000.20241013130934.filter_absorption_photometer...3mo.1h.IT06L_Magee_AE33_CMN_NRT.IT06L_eBC_v1.lev3b.nc',
    'IT0009R.20240713170000.20241013130604.filter_absorption_photometer...3mo.1h.IT06L_Magee_AE33_CMN_NRT.IT06L_AE33.lev1.5.nc',
    'IT0004R.20240714050000.20241014043402.nephelometer..pm10.3mo.1h.IT04L_TSI_3563_IPR_dry_NRT.IT04L_scat_coef.lev1.5.nc',
    'IE0011U.20240714210000.20241014043605.aerosol_mass_spectrometer...13w.1h.IE04L_Aerodyne_Q-ACSM_140-133_ACSMdry_NRT.IE04L_Aerodyne_Q-ACSM_.lev1.5.nc',
    'HU0002R.20240806050000.20241014051404.dmps...69d.1h.FI03L_UHEL_DMPS_PUS_01_NRT.FI03L_DMPS_PUS_01.lev1.5.nc',
    'GR0100B.20240714050000.20241014051206.filter_absorption_photometer...3mo.1h.GR05L_Magee_AE33_DEM_dry_NRT.GR05L_eBC_v1.lev3b.nc',
    'GR0100B.20240714050000.20241014050603.filter_absorption_photometer...3mo.1h.GR05L_Magee_AE33_DEM_dry_NRT.GR05L_abs_coef_AE33_v1.lev1.5.nc',
    'FR0030R.20240716010000.20241009084004.aerosol_mass_spectrometer...3mo.1h.FR04L_Aerodyne_Q-ACSM_140-142_NRT.FR01L_Ng_2011.lev1.5.nc',
    'FR0020R.20240716110000.20241014043603.aerosol_mass_spectrometer...3mo.1h.FR01L_Aerodyne_Q-ACSM_140-113_NRT.FR01L_Aerodyne_Q-ACSM_GIF.lev1.5.nc',
    'FR0020R.20240425230000.20240725211404.filter_absorption_photometer...3mo.1h.FR01L_Magee_AE33_SIR_pm10_NRT.FR01L_eBC_v1.lev3b.nc',
    'FR0020R.20240425230000.20240725211002.filter_absorption_photometer...3mo.1h.FR01L_Magee_AE33_SIR_pm10_NRT.FR01L_Magee_AE33_SIR.lev1.5.nc',
    'FI0050R.20240714050000.20241014051228.cpc...3mo.1h.FI03L_CPC_HYY_01_NRT.FI03L_CPC2.lev1.5.nc',
    'FI0050R.20240714050000.20241014051206.dmps...3mo.1h.FI03L_UHEL_SMPS_HYY_NRT.FI03L_TRY_SMPS.lev1.5.nc',
    'FI0038U.20240714060000.20241014051523.dmps...3mo.1h.FI03L_UHEL_DMPS_KUM_01_NRT.FI03L_TRY_TDMPS.lev1.5.nc',
    'FI0038U.20240714050000.20241014051552.cpc...3mo.1h.FI03L_CPC_KUM_01_NRT.FI03L_CPC.lev1.5.nc',
    'FI0023R.20241008080000.20241011211232.filter_absorption_photometer...12h.1h.FI03L_Magee_AE36_VAR_NRT.FI03L_eBC_v1.lev3b.nc',
    'FI0023R.20241008070000.20241011211232.filter_absorption_photometer...86h.1h.FI03L_Magee_AE36_VAR_NRT.FI03L_ae36_ECACv1.lev1.5.nc',
    'FI0023R.20240806090000.20241014051231.nephelometer...10w.1h.FI03L_Ecotech_3000_VAR_NRT.FI03L_scat_coef.lev1.5.nc',
    'FI0023R.20240709230000.20241009200906.dmps...3mo.1h.FI03L_UHEL_DMPS_VAR_02_NRT.FI03L_TRY_TDMPS.lev1.5.nc',
    'FI0023R.20240707170000.20241007170939.cpc...3mo.1h.FI03L_CPC_VAR_01_NRT.FI03L_CPC_20240606.lev1.5.nc',
    'ES0100R.20240714050000.20241014044219.nephelometer...3mo.1h.ES06L_TSI_3563_ARN_NRT.ES06L_scat_coef.lev1.5.nc',
    'ES0020U.20240714050000.20241014044404.nephelometer..aerosol.3mo.1h.ES08L_TSI_3563_UGR_NRT.ES08L_scat_coef.lev1.5.nc',
    'ES0019U.20240714050000.20241014051506.smps...3mo.1h.ES05L_SMPS3082_NRT.ES05L_none.lev1.5.nc',
    'ES0019U.20240703050000.20241003030929.filter_absorption_photometer...3mo.1h.ES05L_Magee_AE33_sn143_NRT.ES05L_eBC_v1.lev3b.nc',
    'ES0019U.20240703050000.20241003030604.filter_absorption_photometer...3mo.1h.ES05L_Magee_AE33_sn143_NRT.ES05L_AE33.lev1.5.nc',
    'DE0044R.20240714050000.20241014043908.aerosol_mass_spectrometer...3mo.1h.DE08L_Aerodyne_Q-ACSM_140-143_NRT.DE08L_Aerodyne_Q-ACSM_MEL.lev1.5.nc',
    'DE0043G.20240714050000.20241014051211.nephelometer..pm10.3mo.1h.DE09L_TSI_3563_NRT.DE09L_neph_lev0.lev1.5.nc',
    'DE0043G.20240625170000.20240923124009.smps...3mo.1h.DE09L_SMPS_MOHP_NRT...nc',
    'CZ0004B.20241007000000.20241011210604.filter_absorption_photometer...5d.1h.CZ03L_Magee_AE33_Suchdol_NRT.CZ06L_abs_coef_AE33_v1_ECACv1.lev1.5.nc',
    'CZ0004B.20240711230000.20241011210933.filter_absorption_photometer...3mo.1h.CZ03L_Magee_AE33_Suchdol_NRT.CZ03L_eBC_v1.lev3b.nc',
    'CZ0004B.20240711230000.20241011210604.filter_absorption_photometer...3mo.1h.CZ03L_Magee_AE33_Suchdol_NRT.CZ06L_abs_coef_AE33_v1.lev1.5.nc',
    'CZ0003R.20240701110000.20241001100803.filter_absorption_photometer...3mo.1h.CZ06L_Magee_AE33_Czechglobe_NRT.CZ06L_eBC_v1.lev3b.nc',
    'CZ0003R.20240701110000.20241001100405.filter_absorption_photometer...3mo.1h.CZ06L_Magee_AE33_Czechglobe_NRT.CZ06L_abs_coef_AE33_v1.lev1.5.nc',
    'CH0001G.20240901024100.20241014033305.online_gc..air.6w.1h.CH01L_Agilent_GC-MS-MEDUSA_Medusa-12_JFJ_NRT.CH01L_gc_ms.lev1.5.nc',
    'BG0001R.20240716150000.20241012155111.filter_absorption_photometer..aerosol.3mo.1h.BG02L_GMD_CLAP-3W_BEO_NRT.BG02L_abs_coef.lev1.5.nc',
    'BG0001R.20240714050000.20241014044519.nephelometer..aerosol.3mo.1h.BG02L_TSI_3563_BEO_NRT.BG02L_scat_coef.lev1.5.nc',
    'AT0034G.20240714050000.20241014051204.nephelometer..aerosol.3mo.1h.AT04L_Ecotech_Aurora_3000_NRT.AT04L_neph_control_lev0_0_0_1.lev1.5.nc'
]

## Check if files can be opened

In [3]:
wrong_files = []
correct_files = []
actris_nrt_path = 'https://thredds.nilu.no/thredds/dodsC/actris_nrt'
for file in files:
    variables = []
    try:
        opendap_url = f'{actris_nrt_path}/{file}'
        ds = xr.open_dataset(opendap_url)
        correct_files.append(file)
    except Exception as error:
        wrong_files.append(file)

print(f'Total number of files: {len(files)}')
print(f'OK: {len(correct_files)}')
print(f'Error: {len(wrong_files)}')

Total number of files: 45
OK: 14
Error: 31


## Collect basic information per file

In [4]:
data_dict = {}
all_variables = {}

In [5]:
actris_nrt_path = 'https://thredds.nilu.no/thredds/dodsC/actris_nrt'
for file in correct_files:
    data_dict[file]={}
    variables = []
    opendap_url = f'{actris_nrt_path}/{file}'
    ds = xr.open_dataset(opendap_url)
    station_code = ds.attrs['ebas_station_code']
    for var, data in ds.data_vars.items():
        if 'ebas_component' in data.attrs:
            speci = data.attrs['ebas_component']
            if speci not in variables:
                variables.append(var)
            if speci not in all_variables:
                all_variables[var] = {}
                
    data_dict[file]['variables'] = variables
    data_dict[file]['station_code'] = station_code
    data_dict[file]['start_date'] = ds.time[0].values
    data_dict[file]['end_date'] = ds.time[1].values

In [6]:
data_dict

{'NO0002R.20241002110000.20241004105104.cpc...47h.1h.NO01L_TSI_3750_BIR_NRT.NO01L_CPC_acquisition_lev0_0_0_1.lev1.5.nc': {'variables': ['particle_number_concentration_amean',
   'particle_number_concentration_prec1587',
   'particle_number_concentration_perc8413',
   'pressure',
   'temperature'],
  'station_code': 'NO0002R',
  'start_date': numpy.datetime64('2024-10-02T11:30:00.000000000'),
  'end_date': numpy.datetime64('2024-10-02T12:30:00.000000000')},
 'NO0002R.20241002090000.20241009001220.filter_absorption_photometer...1w.1h.NO01L_Magee_AE33_BIR_dry_NRT.NO01L_abs_coef_AE33_v1.lev1.5.nc': {'variables': ['pressure',
   'temperature',
   'aerosol_absorption_coefficient_amean',
   'aerosol_absorption_coefficient_prec1587',
   'aerosol_absorption_coefficient_perc8413'],
  'station_code': 'NO0002R',
  'start_date': numpy.datetime64('2024-10-02T09:30:00.000000000'),
  'end_date': numpy.datetime64('2024-10-02T10:30:00.000000000')},
 'IT0009R.20240713170000.20241013130934.filter_absorpti

In [7]:
all_variables

{'particle_number_concentration_amean': {},
 'particle_number_concentration_prec1587': {},
 'particle_number_concentration_perc8413': {},
 'pressure': {},
 'temperature': {},
 'aerosol_absorption_coefficient_amean': {},
 'aerosol_absorption_coefficient_prec1587': {},
 'aerosol_absorption_coefficient_perc8413': {},
 'aerosol_absorption_angstrom_exponent': {},
 'equivalent_black_carbon_amean': {},
 'equivalent_black_carbon_prec1587': {},
 'equivalent_black_carbon_perc8413': {},
 'ammonium_ug_N_per_m3_amean': {},
 'ammonium_ug_per_m3_amean': {},
 'ammonium_ug_N_per_m3_uncertainty': {},
 'ammonium_ug_per_m3_uncertainty': {},
 'chloride_amean': {},
 'chloride_uncertainty': {},
 'nitrate_ug_N_per_m3_amean': {},
 'nitrate_ug_per_m3_amean': {},
 'nitrate_ug_N_per_m3_uncertainty': {},
 'nitrate_ug_per_m3_uncertainty': {},
 'organic_mass_amean': {},
 'organic_mass_uncertainty': {},
 'sulphate_total_ug_S_per_m3_amean': {},
 'sulphate_total_ug_per_m3_amean': {},
 'sulphate_total_ug_S_per_m3_uncert

## Get files per variable

In [8]:
for var in all_variables.keys():
    variable_files = []
    for file in data_dict.keys():
        if var in data_dict[file]['variables']:
            variable_files.append(file)
    all_variables[var]['files'] = variable_files

In [9]:
all_variables

{'particle_number_concentration_amean': {'files': ['NO0002R.20241002110000.20241004105104.cpc...47h.1h.NO01L_TSI_3750_BIR_NRT.NO01L_CPC_acquisition_lev0_0_0_1.lev1.5.nc',
   'FI0023R.20240707170000.20241007170939.cpc...3mo.1h.FI03L_CPC_VAR_01_NRT.FI03L_CPC_20240606.lev1.5.nc']},
 'particle_number_concentration_prec1587': {'files': ['NO0002R.20241002110000.20241004105104.cpc...47h.1h.NO01L_TSI_3750_BIR_NRT.NO01L_CPC_acquisition_lev0_0_0_1.lev1.5.nc',
   'FI0023R.20240707170000.20241007170939.cpc...3mo.1h.FI03L_CPC_VAR_01_NRT.FI03L_CPC_20240606.lev1.5.nc']},
 'particle_number_concentration_perc8413': {'files': ['NO0002R.20241002110000.20241004105104.cpc...47h.1h.NO01L_TSI_3750_BIR_NRT.NO01L_CPC_acquisition_lev0_0_0_1.lev1.5.nc',
   'FI0023R.20240707170000.20241007170939.cpc...3mo.1h.FI03L_CPC_VAR_01_NRT.FI03L_CPC_20240606.lev1.5.nc']},
 'pressure': {'files': ['NO0002R.20241002110000.20241004105104.cpc...47h.1h.NO01L_TSI_3750_BIR_NRT.NO01L_CPC_acquisition_lev0_0_0_1.lev1.5.nc',
   'NO0002

## Test combination

In [10]:
file_1 = all_variables['particle_number_concentration_amean']['files'][0]
file_2 = all_variables['particle_number_concentration_amean']['files'][1]

In [11]:
ds_1 = xr.open_dataset(f'{actris_nrt_path}/{file_1}')
ds_1

<xarray.Dataset> Size: 7kB
Dimensions:                                              (time: 47, tbnds: 2,
                                                          metadata_time: 1,
                                                          Location: 1,
                                                          pressure_qc_flags: 2,
                                                          temperature_qc_flags: 2,
                                                          particle_number_concentration_amean_qc_flags: 2,
                                                          particle_number_concentration_prec1587_qc_flags: 2,
                                                          particle_number_concentration_perc8413_qc_flags: 2)
Coordinates:
  * time                                                 (time) datetime64[ns] 376B ...
  * metadata_time                                        (metadata_time) datetime64[ns] 8B ...
  * Location                                             (Location) |S64 64B ...
Dimensions without coordinates: tbnds, pressure_qc_flags, temperature_qc_flags,
                                particle_number_concentration_amean_qc_flags,
                                particle_number_concentration_prec1587_qc_flags,
                                particle_number_concentration_perc8413_qc_flags
Data variables: (12/17)
    time_bnds                                            (time, tbnds) datetime64[ns] 752B ...
    metadata_time_bnds                                   (metadata_time, tbnds) datetime64[ns] 16B ...
    pressure_qc                                          (Location, pressure_qc_flags, time) float64 752B ...
    pressure_ebasmetadata                                (Location, metadata_time) |S64 64B ...
    temperature_qc                                       (Location, temperature_qc_flags, time) float64 752B ...
    temperature_ebasmetadata                             (Location, metadata_time) |S64 64B ...
    ...                                                   ...
    particle_number_concentration_prec1587_ebasmetadata  (metadata_time) |S64 64B ...
    particle_number_concentration_perc8413               (time) float64 376B ...
    particle_number_concentration_perc8413_qc            (particle_number_concentration_perc8413_qc_flags, time) float64 752B ...
    particle_number_concentration_perc8413_ebasmetadata  (metadata_time) |S64 64B ...
    pressure                                             (Location, time) float64 376B ...
    temperature                                          (Location, time) float64 376B ...
Attributes: (12/111)
    Conventions:                                   CF-1.8, ACDD-1.3
    featureType:                                   timeSeries
    title:                                         Particle_number_concentrat...
    keywords:                                      Birkenes II, pm10, ACTRIS_...
    id:                                            NO0002R.20241002110000.202...
    naming_authority:                              EBAS
    ...                                            ...
    geospatial_lat_units:                          degrees_north
    geospatial_lon_units:                          degrees_east
    comment:                                       {\n    "Data definition": ...
    standard_name_vocabulary:                      CF-1.7, ACDD-1.3
    history:                                       None
    creator_url:                                   ebas.nilu.no

In [12]:
ds_1 = ds_1.expand_dims(dim={"station": [0]})
ds_1

<xarray.Dataset> Size: 7kB
Dimensions:                                              (station: 1, time: 47,
                                                          tbnds: 2,
                                                          metadata_time: 1,
                                                          Location: 1,
                                                          pressure_qc_flags: 2,
                                                          temperature_qc_flags: 2,
                                                          particle_number_concentration_amean_qc_flags: 2,
                                                          particle_number_concentration_prec1587_qc_flags: 2,
                                                          particle_number_concentration_perc8413_qc_flags: 2)
Coordinates:
  * station                                              (station) int64 8B 0
  * time                                                 (time) datetime64[ns] 376B ...
  * metadata_time                                        (metadata_time) datetime64[ns] 8B ...
  * Location                                             (Location) |S64 64B ...
Dimensions without coordinates: tbnds, pressure_qc_flags, temperature_qc_flags,
                                particle_number_concentration_amean_qc_flags,
                                particle_number_concentration_prec1587_qc_flags,
                                particle_number_concentration_perc8413_qc_flags
Data variables: (12/17)
    time_bnds                                            (station, time, tbnds) datetime64[ns] 752B ...
    metadata_time_bnds                                   (station, metadata_time, tbnds) datetime64[ns] 16B ...
    pressure_qc                                          (station, Location, pressure_qc_flags, time) float64 752B ...
    pressure_ebasmetadata                                (station, Location, metadata_time) |S64 64B ...
    temperature_qc                                       (station, Location, temperature_qc_flags, time) float64 752B ...
    temperature_ebasmetadata                             (station, Location, metadata_time) |S64 64B ...
    ...                                                   ...
    particle_number_concentration_prec1587_ebasmetadata  (station, metadata_time) |S64 64B ...
    particle_number_concentration_perc8413               (station, time) float64 376B ...
    particle_number_concentration_perc8413_qc            (station, particle_number_concentration_perc8413_qc_flags, time) float64 752B ...
    particle_number_concentration_perc8413_ebasmetadata  (station, metadata_time) |S64 64B ...
    pressure                                             (station, Location, time) float64 376B ...
    temperature                                          (station, Location, time) float64 376B ...
Attributes: (12/111)
    Conventions:                                   CF-1.8, ACDD-1.3
    featureType:                                   timeSeries
    title:                                         Particle_number_concentrat...
    keywords:                                      Birkenes II, pm10, ACTRIS_...
    id:                                            NO0002R.20241002110000.202...
    naming_authority:                              EBAS
    ...                                            ...
    geospatial_lat_units:                          degrees_north
    geospatial_lon_units:                          degrees_east
    comment:                                       {\n    "Data definition": ...
    standard_name_vocabulary:                      CF-1.7, ACDD-1.3
    history:                                       None
    creator_url:                                   ebas.nilu.no

In [13]:
ds_2 = xr.open_dataset(f'{actris_nrt_path}/{file_2}')
ds_2

<xarray.Dataset> Size: 225kB
Dimensions:                                              (time: 2163, tbnds: 2,
                                                          metadata_time: 1,
                                                          Location: 1,
                                                          pressure_qc_flags: 1,
                                                          temperature_qc_flags: 1,
                                                          particle_number_concentration_amean_qc_flags: 1,
                                                          particle_number_concentration_prec1587_qc_flags: 1,
                                                          particle_number_concentration_perc8413_qc_flags: 1)
Coordinates:
  * time                                                 (time) datetime64[ns] 17kB ...
  * metadata_time                                        (metadata_time) datetime64[ns] 8B ...
  * Location                                             (Location) |S64 64B ...
Dimensions without coordinates: tbnds, pressure_qc_flags, temperature_qc_flags,
                                particle_number_concentration_amean_qc_flags,
                                particle_number_concentration_prec1587_qc_flags,
                                particle_number_concentration_perc8413_qc_flags
Data variables: (12/17)
    time_bnds                                            (time, tbnds) datetime64[ns] 35kB ...
    metadata_time_bnds                                   (metadata_time, tbnds) datetime64[ns] 16B ...
    pressure_qc                                          (Location, pressure_qc_flags, time) float64 17kB ...
    pressure_ebasmetadata                                (Location, metadata_time) |S64 64B ...
    temperature_qc                                       (Location, temperature_qc_flags, time) float64 17kB ...
    temperature_ebasmetadata                             (Location, metadata_time) |S64 64B ...
    ...                                                   ...
    particle_number_concentration_prec1587_ebasmetadata  (metadata_time) |S64 64B ...
    particle_number_concentration_perc8413               (time) float64 17kB ...
    particle_number_concentration_perc8413_qc            (particle_number_concentration_perc8413_qc_flags, time) float64 17kB ...
    particle_number_concentration_perc8413_ebasmetadata  (metadata_time) |S64 64B ...
    pressure                                             (Location, time) float64 17kB ...
    temperature                                          (Location, time) float64 17kB ...
Attributes: (12/103)
    Conventions:                                   CF-1.8, ACDD-1.3
    featureType:                                   timeSeries
    title:                                         Particle_number_concentrat...
    keywords:                                      pm10, ACTRIS_NRT, FI0023R,...
    id:                                            FI0023R.20240707170000.202...
    naming_authority:                              EBAS
    ...                                            ...
    geospatial_lat_units:                          degrees_north
    geospatial_lon_units:                          degrees_east
    comment:                                       {\n    "Data definition": ...
    standard_name_vocabulary:                      CF-1.7, ACDD-1.3
    history:                                       None
    creator_url:                                   ebas.nilu.no

In [14]:
ds_2 = ds_2.expand_dims(dim={"station": [1]})
ds_2

<xarray.Dataset> Size: 225kB
Dimensions:                                              (station: 1,
                                                          time: 2163, tbnds: 2,
                                                          metadata_time: 1,
                                                          Location: 1,
                                                          pressure_qc_flags: 1,
                                                          temperature_qc_flags: 1,
                                                          particle_number_concentration_amean_qc_flags: 1,
                                                          particle_number_concentration_prec1587_qc_flags: 1,
                                                          particle_number_concentration_perc8413_qc_flags: 1)
Coordinates:
  * station                                              (station) int64 8B 1
  * time                                                 (time) datetime64[ns] 17kB ...
  * metadata_time                                        (metadata_time) datetime64[ns] 8B ...
  * Location                                             (Location) |S64 64B ...
Dimensions without coordinates: tbnds, pressure_qc_flags, temperature_qc_flags,
                                particle_number_concentration_amean_qc_flags,
                                particle_number_concentration_prec1587_qc_flags,
                                particle_number_concentration_perc8413_qc_flags
Data variables: (12/17)
    time_bnds                                            (station, time, tbnds) datetime64[ns] 35kB ...
    metadata_time_bnds                                   (station, metadata_time, tbnds) datetime64[ns] 16B ...
    pressure_qc                                          (station, Location, pressure_qc_flags, time) float64 17kB ...
    pressure_ebasmetadata                                (station, Location, metadata_time) |S64 64B ...
    temperature_qc                                       (station, Location, temperature_qc_flags, time) float64 17kB ...
    temperature_ebasmetadata                             (station, Location, metadata_time) |S64 64B ...
    ...                                                   ...
    particle_number_concentration_prec1587_ebasmetadata  (station, metadata_time) |S64 64B ...
    particle_number_concentration_perc8413               (station, time) float64 17kB ...
    particle_number_concentration_perc8413_qc            (station, particle_number_concentration_perc8413_qc_flags, time) float64 17kB ...
    particle_number_concentration_perc8413_ebasmetadata  (station, metadata_time) |S64 64B ...
    pressure                                             (station, Location, time) float64 17kB ...
    temperature                                          (station, Location, time) float64 17kB ...
Attributes: (12/103)
    Conventions:                                   CF-1.8, ACDD-1.3
    featureType:                                   timeSeries
    title:                                         Particle_number_concentrat...
    keywords:                                      pm10, ACTRIS_NRT, FI0023R,...
    id:                                            FI0023R.20240707170000.202...
    naming_authority:                              EBAS
    ...                                            ...
    geospatial_lat_units:                          degrees_north
    geospatial_lon_units:                          degrees_east
    comment:                                       {\n    "Data definition": ...
    standard_name_vocabulary:                      CF-1.7, ACDD-1.3
    history:                                       None
    creator_url:                                   ebas.nilu.no

In [15]:
combined_ds_list = [ds_1['particle_number_concentration_amean'], 
                    ds_2['particle_number_concentration_amean']]

In [16]:
combined_ds = xr.concat(combined_ds_list, 
                        dim='station', 
                        combine_attrs='drop_conflicts')
combined_ds = combined_ds.to_dataset()
combined_ds['ebas_station_code'] = xr.Variable(data=['NO0002R', 'FI0023R'], dims=('station'))
combined_ds

<xarray.Dataset> Size: 52kB
Dimensions:                              (time: 2164, station: 2)
Coordinates:
  * time                                 (time) datetime64[ns] 17kB 2024-07-0...
  * station                              (station) int64 16B 0 1
Data variables:
    particle_number_concentration_amean  (station, time) float64 35kB nan ......
    ebas_station_code                    (station) <U7 56B 'NO0002R' 'FI0023R'

In [17]:
ebas_keys = [key for key in ds_1.attrs.keys() if key.startswith('ebas_station')]
ebas_keys

['ebas_station_code',
 'ebas_station_name',
 'ebas_station_wdca_id',
 'ebas_station_gaw_id',
 'ebas_station_gaw_name',
 'ebas_station_other_ids',
 'ebas_station_land_use',
 'ebas_station_setting',
 'ebas_station_gaw_type',
 'ebas_station_wmo_region',
 'ebas_station_latitude',
 'ebas_station_longitude',
 'ebas_station_altitude']

## Preprocessing of files

In [18]:
# Define metadata keys, inspired in first dataset (TODO: Check all available metadata fields)
metadata_keys = ['ebas_station_code','ebas_station_name',
                 'ebas_station_wdca_id', 'ebas_station_gaw_id',
                 'ebas_station_gaw_name', 'ebas_station_other_ids',
                 'ebas_station_land_use', 'ebas_station_setting',
                 'ebas_station_gaw_type', 'ebas_station_wmo_region',
                 'ebas_station_latitude', 'ebas_station_longitude',
                 'ebas_station_altitude']

# open all files for certain variable, extract and join
resolution = 'hourly'
for var in all_variables:

    # combine datasets that have the same variable
    combined_ds_list = []
    metadata = {}
    for i, file in enumerate(all_variables[var]['files']):
        # open file
        opendap_url = f'{actris_nrt_path}/{file}'
        ds = xr.open_dataset(opendap_url)

        # save metadata
        for ebas_key in metadata_keys:
            if ebas_key not in metadata.keys():
                metadata[ebas_key] = []
            if ebas_key not in ds.attrs.keys():
                metadata[ebas_key].append(np.nan)
            else:
                metadata[ebas_key].append(ds.attrs[ebas_key])
        
        # assign station code as dimension
        ds = ds.expand_dims(dim={'station': [i]})

        # select data for that variable only
        ds_var = ds[var]

        # append modified dataset to list
        combined_ds_list.append(ds_var)

    # combine and create new dataset
    combined_ds = xr.concat(combined_ds_list, 
                            dim='station', 
                            combine_attrs='drop_conflicts').to_dataset()

    # add metadata
    for key in metadata.keys():
        combined_ds[key] = xr.Variable(data=metadata[key], dims=('station'))
    
    # save data per year and month
    path = f'/home/avilanov/data/providentia/obs/nonghost/actris/nrt/{resolution}/{var}'
    if not os.path.isdir(path):
        os.makedirs(path, exist_ok=True)
    for year, ds_year in combined_ds.groupby('time.year'):
        for month, ds_month in ds_year.groupby('time.month'):
            filename = f"{path}/{var}_{year}{month:02d}.nc"
            combined_ds_yearmonth = combined_ds.sel(time=f"{year}-{month:02d}")
            combined_ds_yearmonth.to_netcdf(filename)
            print(f"Saved: {filename}")

Saved: /home/avilanov/data/providentia/obs/nonghost/actris/nrt/hourly/particle_number_concentration_amean/particle_number_concentration_amean_202407.nc
Saved: /home/avilanov/data/providentia/obs/nonghost/actris/nrt/hourly/particle_number_concentration_amean/particle_number_concentration_amean_202408.nc
Saved: /home/avilanov/data/providentia/obs/nonghost/actris/nrt/hourly/particle_number_concentration_amean/particle_number_concentration_amean_202409.nc
Saved: /home/avilanov/data/providentia/obs/nonghost/actris/nrt/hourly/particle_number_concentration_amean/particle_number_concentration_amean_202410.nc
Saved: /home/avilanov/data/providentia/obs/nonghost/actris/nrt/hourly/particle_number_concentration_prec1587/particle_number_concentration_prec1587_202407.nc
Saved: /home/avilanov/data/providentia/obs/nonghost/actris/nrt/hourly/particle_number_concentration_prec1587/particle_number_concentration_prec1587_202408.nc
Saved: /home/avilanov/data/providentia/obs/nonghost/actris/nrt/hourly/partic

## Check new files

In [19]:
xr.open_dataset('/home/avilanov/data/providentia/obs/nonghost/actris/nrt/hourly/aerosol_absorption_angstrom_exponent/aerosol_absorption_angstrom_exponent_202407.nc')

<xarray.Dataset> Size: 33kB
Dimensions:                               (time: 743, Wavelength_1: 1,
                                           Wavelength_2: 1, station: 4)
Coordinates:
  * time                                  (time) datetime64[ns] 6kB 2024-07-0...
  * Wavelength_1                          (Wavelength_1) float64 8B 590.0
  * Wavelength_2                          (Wavelength_2) float64 8B 660.0
  * station                               (station) int64 32B 0 1 2 3
Data variables: (12/14)
    aerosol_absorption_angstrom_exponent  (station, Wavelength_1, Wavelength_2, time) float64 24kB ...
    ebas_station_code                     (station) <U7 112B ...
    ebas_station_name                     (station) <U38 608B ...
    ebas_station_wdca_id                  (station) <U11 176B ...
    ebas_station_gaw_id                   (station) <U3 48B ...
    ebas_station_gaw_name                 (station) <U20 320B ...
    ...                                    ...
    ebas_station_setting                  (station) <U21 336B ...
    ebas_station_gaw_type                 (station) <U3 48B ...
    ebas_station_wmo_region               (station) <U3 48B ...
    ebas_station_latitude                 (station) <U17 272B ...
    ebas_station_longitude                (station) <U18 288B ...
    ebas_station_altitude                 (station) <U8 128B ...

In [20]:
ghost_data = xr.open_dataset('/home/avilanov/data/providentia/obs/ghost/AERONET_v3_lev1.5/1.5/hourly/ae440-870aero/ae440-870aero_201601.nc')

In [21]:
ghost_data

<xarray.Dataset> Size: 264MB
Dimensions:                                                           (
                                                                       station: 316,
                                                                       time: 744,
                                                                       N_flag_codes: 186,
                                                                       N_qa_codes: 79)
Coordinates:
  * time                                                              (time) datetime64[ns] 6kB ...
Dimensions without coordinates: station, N_flag_codes, N_qa_codes
Data variables: (12/183)
    ASTER_v3_altitude                                                 (station) float32 1kB ...
    EDGAR_v4.3.2_annual_average_BC_emissions                          (station) float32 1kB ...
    EDGAR_v4.3.2_annual_average_CO_emissions                          (station) float32 1kB ...
    EDGAR_v4.3.2_annual_average_NH3_emissions                         (station) float32 1kB ...
    EDGAR_v4.3.2_annual_average_NMVOC_emissions                       (station) float32 1kB ...
    EDGAR_v4.3.2_annual_average_NOx_emissions                         (station) float32 1kB ...
    ...                                                                ...
    station_timezone                                                  (station) <U30 38kB ...
    street_type                                                       (station) <U3 4kB ...
    street_width                                                      (station) float32 1kB ...
    terrain                                                           (station) <U3 4kB ...
    vertical_datum                                                    (station) <U22 28kB ...
    weekday_weekend_code                                              (station, time) uint8 235kB ...
Attributes:
    title:             Surface angstrom exponent between 440 and 870 nm data ...
    institution:       Barcelona Supercomputing Center
    creator_name:      Dene R. Bowdalo
    creator_email:     dene.bowdalo@bsc.es
    source:            Observations
    version:           1.5
    application_area:  Monitoring atmospheric composition
    domain:            Atmosphere
    observed_layer:    Total atmospheric column
    data_licence:      BSD-3-Clause. Copyright 2023 Dene Bowdalo.